In [75]:
from bs4 import BeautifulSoup
import urllib.request as request
import urllib.parse as urlparse
import re
import json
import codecs
reader = codecs.getreader('utf-8')

In [36]:
def is_num(text):
    return re.search('^(0|[1-9][0-9]*)$',text)

In [37]:
baseurl = 'https://www.govtrack.us/data/congress'
page = request.urlopen(baseurl)
soup = BeautifulSoup(page.read(),'lxml')

In [55]:
for link in soup.findAll('a'):
    href = link.get('href')
    text = link.getText().strip('/')
    if is_num(text) and int(text)>110:
        # go in
        url = urlparse.urljoin(baseurl,href)
        print(url)
        url = urlparse.urljoin(url,'votes/')
        page = request.urlopen(url)
        bs = BeautifulSoup(page.read(),'lxml')
        for year_link in bs.findAll('a'):
            year = link.getText().strip('/')
            print(year)
            if is_num(year) and int(year) > 1990:
                print(year)

https://www.govtrack.us/data/111/


HTTPError: HTTP Error 404: Not Found

In [85]:
baseurl = 'https://www.govtrack.us/data/congress/'
crange = range(104,115)
for c in crange:
    url = baseurl+str(c)+'/votes/'
    page = request.urlopen(url)
    bs = BeautifulSoup(page.read(),'lxml')
    for link in bs.findAll('a'):
        href = link.get('href')
        text = link.getText().strip('/')
        if not is_num(text): continue
        suburl = urlparse.urljoin(url,href)
        subpage = request.urlopen(suburl)
        subbs = BeautifulSoup(subpage.read(),'lxml')
        for sublink in subbs.findAll('a'):
            subhref = sublink.get('href')
            subtext = sublink.getText().strip('/')
            # if looking for senate use 's'
            # if looking for house use 'h'
            if 's' in subtext:
                # get the data in json
                dataurl = suburl+subhref+'data.json'
                datapage = request.urlopen(dataurl)
                data = json.load(reader(datapage))
                
                session = dataurl.split('/')[-2]
                year = dataurl.split('/')[-3]
                num = dataurl.split('/')[-5]
                outfile = open('data/'+year+'_'+num+'_'+session+'.json','w')
                json.dump(data, outfile)

In [79]:
tester = 'https://www.govtrack.us/data/congress/113/votes/2013/s1/data.json'
page = request.urlopen(tester)
json.load(reader(page))

{'bill': {'congress': 113,
  'number': 15,
  'title': 'A resolution to improve procedures for the consideration of legislation and nominations in the Senate.',
  'type': 'sres'},
 'category': 'passage',
 'chamber': 's',
 'congress': 113,
 'date': '2013-01-24T19:54:00-05:00',
 'number': 1,
 'question': 'On the Resolution S.Res. 15',
 'record_modified': '2013-01-24T20:38:00-05:00',
 'requires': '3/5',
 'result': 'Resolution Agreed to',
 'result_text': 'Resolution Agreed to (78-16, 3/5 majority required)',
 'session': '2013',
 'source_url': 'http://www.senate.gov/legislative/LIS/roll_call_votes/vote1131/vote_113_1_00001.xml',
 'subject': 'S. Res. 15',
 'type': 'On the Resolution',
 'updated_at': '2014-06-18T11:22:45-04:00',
 'vote_id': 's1-113.2013',
 'votes': {'Nay': [{'display_name': 'Crapo (R-ID)',
    'first_name': 'Mike',
    'id': 'S266',
    'last_name': 'Crapo',
    'party': 'R',
    'state': 'ID'},
   {'display_name': 'Cruz (R-TX)',
    'first_name': 'Ted',
    'id': 'S355',
    

In [70]:
page.read()

b''